In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Deep Neural Networks 
## Session 10a:Lecture

## Working with Tensorflow 
- One hidden layer
- Make_moon

</p><img src='../../prasami_images/prasami_color_tutorials_small.png' width='400' alt="By Pramod Sharma : pramod.sharma@prasami.com" align="left"/>

In [ ]:
# Lets import some libraries
import os

import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

from sklearn import datasets

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

from sklearn.preprocessing import StandardScaler

import tensorflow as tf

%matplotlib inline

In [ ]:
# Some basic parameters

inpDir = '../../input' # location where input data is stored
outDir = '../output' # location to store outputs
modelDir = '../models'
subDir = 'make_moon'

RANDOM_STATE = 24 # for initialization ----- REMEMBER: to remove at the time of promotion to production

np.random.seed(RANDOM_STATE) # Set Random Seed for reproducible  results
tf.random.set_seed(RANDOM_STATE) # setting for Tensorflow as well


EPOCHS = 100 # number of epochs
ALPHA = 0.01 # learning rate

#BATCH_SIZE = 256 
NUM_SAMPLE = 1280
NOISE = 0.2
TEST_SIZE = 0.2

# Set parameters for decoration of plots
params = {'legend.fontsize' : 'large',
          'figure.figsize'  : (15,15),
          'axes.labelsize'  : 'x-large',
          'axes.titlesize'  :'x-large',
          'xtick.labelsize' :'large',
          'ytick.labelsize' :'large',
         }

CMAP = plt.cm.brg

plt.rcParams.update(params) # update rcParams

### Settings so that Tensorflow can not Hog all the GPU memory

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU') 

if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Load make_moon

In [ ]:
X, y = datasets.make_moons(n_samples=NUM_SAMPLE, noise=NOISE, random_state=RANDOM_STATE)
X.shape, y.shape

In [ ]:
#  Split the data in training and test sets to measure performance of the model.
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=TEST_SIZE,
                                                    stratify=y,
                                                    random_state=RANDOM_STATE )

print (X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
sc = StandardScaler()

X_train = sc.fit_transform(X_train)

X_test = sc.transform(X_test)

In [ ]:
X_train.shape

## Prepare model

Build the `tf.keras.Sequential` model by stacking layers from `tf.keras.layers.Dense`. Activations can be given as string or from `tf.keras.activations.tanh` 

In [ ]:
# Define Model Calling class Sequential and pass all layers as a list


## Reference
<p style="font-family: Arial; font-size:1.2em;color:black;">
Details of <a href = "https://www.tensorflow.org/api_docs/python/tf/keras/Sequential">Sequential Model</a>. Sequential provides training and predict features on this model.
</p>

### Functional API

<p style="font-family: Arial; font-size:1.2em;color:black;">You can also instantiate a Model with the <strong>"Functional API"</strong>, where you start from Input, you chain layer calls to specify the model's forward pass, and finally you create your model from inputs and outputs:

>import tensorflow as tf

>inputs = tf.keras.Input(shape=(2,))
    
>x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)

>outputs = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(x)

>model = tf.keras.Model(inputs=inputs, outputs=outputs)

    
<p style="font-family: Arial; font-size:1.2em;color:black;">You can also extend the model class, if love to live by sword!!
</p>

<p style="font-family: Arial; font-size:1.2em;color:black;">
    For each example the model returns a vector of "logits" or "log-odds" scores, one for each class.
</p>

<p style="font-family: Arial; font-size:1.2em;color:black;">
    <b>Unit Test</b>: Predict using untrained model.
</p>

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [ ]:
'''
Number of parameters:
    layer 1 : 2*4 +4 = 12 
    layer 2 : 4*4 + 2 = 10
We are not locking any layer hence all parameter are trainable    
'''

model.summary()

## Game Changer

`Softmax` function is prone to two issues: overflow and underflow.

- <b>Overflow</b> occurs when very large numbers are being fed.
- <b>Underflow</b> occurs when very small numbers (near zero) are being fed.

There are work arounds too, but, TF has its own approach.<br>
    
Intuition being that every system (32 / 64 bit precision) has limits in terms of largest or smallest value it can accurately process.

It also leads to the vanishing gradient in exponential functions. As you go higher/lower it  will overflow and the gradient will be `zero` irrespective of actually it being a `non-zero`.

## Lose Function

Our loss function is crossentropy as there are two or more label classes. The labels are provided using one-hot representation. We will use CategoricalCrossentropy loss. 
    
>y_true = [ 0, 2 ] <b>Sparse Representation</b><br>
>y_true = [ [1      0    0], [ 0    0   1 ] <b>Dense Representation</b><br>
>y_pred = [ [0.05, 0.95, 0], [0.1, 0.1, 0.8] ]


In case, you have not one-hot-encoded use `losses.SparseCategoricalCrossentropy`. It takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [ ]:
import sys

sys.float_info.max

In [ ]:
sys.float_info.min

In [ ]:
np.log(sys.float_info.max)

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy ( from_logits = True)

In [ ]:
y_train[:1]

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

## Compile model
Using `model.compile`, specify the training configuration such as:
- Loss function to minimize
- Optimizer `'SGD'`
- List of metrics to monitor `'accuracy'`

## pass model
Call `fit(X, y, validation_data=[X_test, y_test], epochs=EPOCHS)` , which will train the model, and repeatedly iterating over the entire dataset for a given number of `epochs`. Pass some validation for monitoring validation loss and metrics at the end of each epoch.

## Evaluate the model
In case we want to separately evaluate this model, we have `evaluate` function.

In [ ]:
model.evaluate ( X_test,  y_test, verbose=2)

In [ ]:
hist_df = pd.DataFrame(history.history)
hist_df.head()

In [ ]:
fig, axes = plt.subplots(1,2 , figsize = (15,6))

ax = axes[0]

hist_df.plot(y = ['loss', 'val_loss'], ax = ax, colormap=CMAP)

ax.set_xlabel("Epochs")
ax.set_ylabel('Loss')
ax.set_title('Errors')
ax.grid();
ax = axes[1]

hist_df.plot( y = ['accuracy', 'val_accuracy'], ax = ax, colormap=CMAP)

ax.set_xlabel("Epochs")
ax.set_ylabel('Accuracy')
ax.grid()
ax.legend(loc = 'lower left')

plt.tight_layout()

## Predictions
Build a prediction model using `tf.keras.Sequential` again and add `tf.keras.layers.Softmax` to model.

Make predictions and print classification reports and confusion matrix.

In [ ]:
print(classification_report(y_train, np.argmax(y_pred, axis = 1)))

In [ ]:
y_pred = probability_model(X_test).numpy()

print('Accuracy score on Test Data :{:.5f}'.format(accuracy_score(y_test, 
                                                                  np.argmax(y_pred, axis = 1))))

In [ ]:
print(classification_report(y_test, 
                            np.argmax(y_pred, axis = 1)))

In [ ]:
cm  = confusion_matrix(y_test, np.argmax(y_pred, axis = 1))

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=[0,1])

fig, ax = plt.subplots(figsize = (4,4))

disp.plot(ax = ax, cmap = 'Blues', xticks_rotation = 'vertical', colorbar=False)

plt.show();

In [ ]:
hist_df[hist_df['val_loss'] == hist_df['val_loss'].min()]